In [ ]:
Пайплайн
========
Часть 2.
1. Авторизуется в Google Photos
2. Подключается к Google Drive, забирая все ссылки из указанной папки 
и сохраняет результат в google_drive_links.csv
3. Берет ссылки из google_drive_links.csv, подставляет их в photos.geojson 
и сохраняет в тот же файл photos.geojson.

In [ ]:
from pydrive.auth import GoogleAuth
gauth = GoogleAuth()
gauth.LocalWebserverAuth()

In [ ]:
# Вот полный скрипт, который автоматически формирует google_drive_links.csv, подключаясь к Google Drive, 
# забирая все ссылки из указанной папки, и сохраняет результат в CSV-файл.

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build_from_document
import pandas as pd
import os
from tqdm import tqdm

# === Настройки ===
SCOPES = ['https://www.googleapis.com/auth/photoslibrary.readonly']
FOLDER_ID = '1em81MElkxnaue5r92e9hPGezgOuw4nRL'
ALBUM_NAMES = ["PhotoMap 2022-2025", "PhotoMap 2019-2021", "PhotoMap 2017-2019", "PhotoMap 2014-2016"]
OUTPUT_CSV = 'csv/google_drive_links.csv'

# === 🌐 Авторизация Google Photos и Google Drive
gauth = GoogleAuth()
gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)

flow = InstalledAppFlow.from_client_secrets_file('client_secrets.json', SCOPES)
credentials = flow.run_local_server(port=0)

# === Загрузка Discovery документа
with open("photoslibrary_v1_discovery.json", "r") as f:
    discovery_doc = f.read()
service = build_from_document(discovery_doc, credentials=credentials)

# === 🔄 Получаем список файлов из Google Drive
print("🔍 Получаем список файлов из Google Drive...")
file_list = drive.ListFile({'q': f"'{FOLDER_ID}' in parents and trashed=false"}).GetList()

drive_data = {
    'filename': [file['title'] for file in file_list],
    'file_id': [file['id'] for file in file_list],
    'link': [f"https://drive.google.com/uc?export=view&id={file['id']}" for file in file_list],
    'description': ['No Description'] * len(file_list)  # УНИФИЦИРОВАЛИ!
}

df_links = pd.DataFrame(drive_data)
df_links.set_index('filename', inplace=True)

# === 🔍 Поиск нужных альбомов ===
print("🔍 Поиск альбомов в Google Photos...")
albums = []
nextPageToken = None

while True:
    response = service.albums().list(pageSize=50, pageToken=nextPageToken).execute()
    albums.extend(response.get('albums', []))
    nextPageToken = response.get('nextPageToken')
    if not nextPageToken:
        break

target_albums = {album['title']: album['id'] for album in albums if album['title'] in ALBUM_NAMES}

# === 💾 Обновляем DataFrame с данными из Google Photos
for album_name, album_id in target_albums.items():
    print(f"\n📂 Альбом: {album_name}")
    nextPageToken = None
    
    with tqdm(desc=f"Загрузка из {album_name}", unit="фото", leave=True) as pbar:
        while True:
            results = service.mediaItems().search(
                body={"albumId": album_id, "pageSize": 100, "pageToken": nextPageToken}
            ).execute()

            items = results.get('mediaItems', [])
            nextPageToken = results.get('nextPageToken')

            if not items:
                break

            for item in items:
                description = item.get('description', 'No Description')
                filename = item.get('filename', 'No Filename')
                
                if filename in df_links.index:
                    df_links.at[filename, 'description'] = description

            # === Динамическое обновление прогресса ===
            pbar.update(len(items))

            if not nextPageToken:
                break

# === 💾 Сохранение в CSV
os.makedirs('csv', exist_ok=True)
df_links.to_csv(OUTPUT_CSV)
print(f"\n✅ Ссылки и описания успешно сохранены в {OUTPUT_CSV}")

In [ ]:
# Вот полный скрипт, который берет ссылки из google_drive_links.csv, 
# подставляет их в photos.geojson и сохраняет в новый файл photos_updated.geojson.

In [ ]:
import pandas as pd
import json

# === 📁 Чтение данных
df_links = pd.read_csv('csv/google_drive_links.csv')

# === 🌐 Загрузка GeoJSON
with open('geojson/photos.geojson', 'r', encoding='utf-8') as file:
    geojson_data = json.load(file)

# === 🔄 Обновление ссылок в GeoJSON с использованием thumbnail и full link
for feature in geojson_data['features']:
    filename = feature['properties']['filename']
    matching_row = df_links[df_links['filename'] == filename]
    
    if not matching_row.empty:
        file_id = matching_row.iloc[0]['file_id']
        full_link = matching_row.iloc[0]['link']
        
        # Обновляем поля
        thumbnail_link = f"https://drive.google.com/thumbnail?id={file_id}"
        feature['properties']['image'] = thumbnail_link
        feature['properties']['fullname'] = full_link

# === 💾 Сохранение обновленного GeoJSON
output_path = 'geojson/photos.geojson'
with open(output_path, 'w', encoding='utf-8') as file:
    json.dump(geojson_data, file, ensure_ascii=False, indent=4)

print(f"✅ Обновленный GeoJSON сохранен в {output_path}")

In [1]:
import pandas as pd
import json

# === 📁 Чтение данных
df_links = pd.read_csv('csv/google_drive_links.csv')

# === 🌐 Загрузка GeoJSON
with open('geojson/photos.geojson', 'r', encoding='utf-8') as file:
    geojson_data = json.load(file)

# === 🔄 Обновление ссылок в GeoJSON с использованием thumbnail, full link и description
for feature in geojson_data['features']:
    filename = feature['properties']['filename']
    matching_row = df_links[df_links['filename'] == filename]
    
    if not matching_row.empty:
        file_id = matching_row.iloc[0]['file_id']
        full_link = matching_row.iloc[0]['link']
        description = matching_row.iloc[0]['description']
        
        # Обновляем поля
        thumbnail_link = f"https://drive.google.com/thumbnail?id={file_id}"
        feature['properties']['image'] = thumbnail_link
        feature['properties']['fullname'] = full_link
        
        # Добавляем Description, если не 'No Description'
        if description != 'No Description':
            feature['properties']['description'] = description

# === 💾 Сохранение обновленного GeoJSON
output_path = 'geojson/photos.geojson'
with open(output_path, 'w', encoding='utf-8') as file:
    json.dump(geojson_data, file, ensure_ascii=False, indent=4)

print(f"✅ Обновленный GeoJSON сохранен в {output_path}")

✅ Обновленный GeoJSON сохранен в geojson/photos.geojson
